In [53]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

1. Загрузите датасет. Описание датасета можно посмотреть здесь.

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [3]:
sms_smam_collection = pd.read_csv("/Users/apple/Jupyter/yandex_coursera/5th_course/NLP_week/recourses/SMSSpamCollection.csv", sep='\t', \
                                 names=['sms_type', 'sms_text'])

In [4]:
sms_smam_collection.head()

,sms_type,sms_text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [26]:
sms_texts = sms_smam_collection['sms_text'].values
sms_labels = (sms_smam_collection['sms_type'] == 'spam').astype(int).values

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.
5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [33]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms_texts)
'%.1f' % cross_val_score(LogisticRegression(random_state=2), X, sms_labels, cv=10, scoring='f1').mean()

'0.9'

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

 "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

In [37]:
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression(random_state=2))]
        )

clf_pipeline.fit(sms_texts, sms_labels)
clf_pipeline.predict([ "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", \
             "FreeMsg: Txt: claim your reward of 3 hours talk time", "Have you visited the last lecture on physics?", \
             "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", \
             "Only 99$"])

array([1, 1, 0, 0, 0])

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [40]:
def sms_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [45]:
print('%.2f' % cross_val_score(sms_classifier(CountVectorizer(ngram_range=(2,2)), LogisticRegression(random_state=2)), \
                         sms_texts, sms_labels, cv=10, scoring='f1').mean())
print('%.2f' % cross_val_score(sms_classifier(CountVectorizer(ngram_range=(3,3)), LogisticRegression(random_state=2)), \
                         sms_texts, sms_labels, cv=10, scoring='f1').mean())
print('%.2f' % cross_val_score(sms_classifier(CountVectorizer(ngram_range=(1,3)), LogisticRegression(random_state=2)), \
                         sms_texts, sms_labels, cv=10, scoring='f1').mean())

0.82
0.72
0.92


8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [51]:
for ngram_range in zip([2, 3, 1], [2, 3, 3]):
    v = CountVectorizer(ngram_range=ngram_range)
    NB_X = v.fit_transform(sms_texts)
    print('%.2f' % cross_val_score(MultinomialNB(), NB_X, sms_labels, cv=10, scoring='f1').mean())

0.65
0.38
0.89


9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [56]:
clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", LogisticRegression(random_state=2))]
        )

cross_val_score(clf_pipeline, sms_texts, sms_labels, cv=10, scoring='f1').mean()

0.8776671468367306